In [349]:
import numpy as np
import scipy as sp
from sympy import *

In [350]:
# def I(BD):
#     BDI=np.zeros(BD.shape)
#     for i in range(BD.shape[0]):
#         for j in range(BD.shape[1]):
#             # print(dblquad(lambda t,x : BD[i,j],0,0.5,0,0.5))
#             BDI[i,j]= dblquad(lambda t,x : BD[i,j],-1,1,-1,1)[0]
#     return BDI

In [351]:
conec=np.array([ [1,2,5,4],[2,3,6,5]])-1
coords=np.array([[0,0],[0.25,0],[0.5,0],[0,0.25],[0.25,0.25],[0.5,0.25] ])

nEle=conec.shape[0]
nodes=coords.shape[0]

dof_seq=np.array([3,4,5,6,9,10,11,12,1,2,7,8])-1
free_dof=8


E=210e6
mu=0.3
h=0.025

In [352]:
#total dofs

def dofs(nodes):
    dof=np.zeros((nodes,2),dtype=int)
    for i in range(nodes):
        dof[i]=[2*i, 2*i+1]
    return dof

dof=dofs(nodes)



In [353]:
#labels
dof_label=np.zeros((nEle,8),dtype=int)
for i in range(nEle):
    pp=dof[conec[i,0]]
    qq=dof[conec[i,1]]
    rr=dof[conec[i,2]]
    ss=dof[conec[i,3]]
    dof_label[i]=np.hstack((pp,qq,rr,ss))#.reshape(1,8)

In [354]:
#elecorrdinates
def elecord(coords,conec):
    elecords=np.zeros((nEle,4,2))
    for i in range(nEle):
        for j in range(conec.shape[1]):
            elecords[i][j]=coords[conec[i,j]]

    return elecords

elecords=elecord(coords,conec)

In [355]:
# def area(BD):
#     BDI=np.array(BD.shape)
#     for i in range(BD.shape[0]):
#         for j in range(BD.shape[-1]):
#             BDI[i,j]=integrate(BD[i,j],(s,-1,1),(t,-1,1))

#     return BDI


In [356]:
#element stiffness for bilinear rectangle

def eleStiffness(elecords,E,mu,h,eleNum,ps=1):
    E=210e6
    coordinates= elecords[eleNum].flatten()
    print(coordinates)
    x1,y1,x2,y2,x3,y3,x4,y4=coordinates
    A1=( x1*(y2-y3) + x2*(y3-y1) +x3 *(y1-y2)/2)
    A2=( x1*(y2-y4) + x3*(y4-y1) +x4 *(y3-y3)/2)
    A=A1+A2
    s=symbols('s')
    t=symbols('t')
    a = (y1*(s-1)+y2*( -1 -s)+y3*(1+s)+y4*(1 -s))/4 
    b = (y1*(t -1)+y2*(1 -t)+y3*(1+t)+y4*( -1 -t))/4 
    c = (x1*(t -1)+x2*(1 -t)+x3*(1+t)+x4*( -1 -t))/4 
    d = (x1*(s -1)+x2*( -1 -s)+x3*(1+s)+x4*(1 -s))/4 

    B1 = np.array([[a*(t -1)/4 -b*(s -1)/4 ,0 ],[0, c*(s -1)/4 -d*(t -1)/4 ],[c*(s -1)/4 -d*(t -1)/4, a*(t -1)/4 -b*(s -1)/4] ])
                    
    B2 = np.array([[a*(1 -t)/4 -b*( -1 -s)/4 ,0],  [0, c*( -1 -s)/4 -d*(1 -t)/4], [ c*( -1 -s)/4 -d*(1 -t)/4, a*(1 -t)/4 -b*( -1 -s)/4]] )

    B3 =  np.array([[a*(t+1)/4 -b*(s+1)/4, 0], [0, c*(s+1)/4 -d*(t+1)/4 ], [c*(s+1)/4-d*(t+1)/4, a*(t+1)/4 -b*(s+1)/4]] )
    
    
    B4 = np.array( [[a*( -1 -t)/4 -b*(1 -s)/4, 0], [0, c*(1 -s)/4 -d*( -1 -t)/4], [c*(1 -s)/4 -d*( -1 -t)/4,a*( -1 -t)/4 -b*(1 -s)/4]])
    
    Bfirst = np.hstack((B1,B2,B3,B4) )
    Jfirst = np.array([[0, 1 -t, t -s ,s -1],[ t -1, 0, s+1,  -s -t ],
                      [s -t,  -s -1, 0, t+1] , [1 -s, s+t , -t -1, 0]])

    M=np.array([x1,x2,x3,x4])
    N=np.array([y1,y2,y3,y4])/8
    
    J=M.T @ Jfirst @ N
    
    B=Bfirst/J
    if ps==1: #plane stresns condition
        D=(E/(1-mu*mu)) * np.array([ [1,mu,0],[mu,1,0],[0,0,(1-mu)/2]])
    elif ps==2: #plane stess condition
        D=(E/((1+mu)*(1-2*mu))) * np.array([ [1-mu,mu,0],[mu,1-mu,0],[ 0,0,(1-2*mu)/2] ])
        
    # print(B.shape)
    BD=J* B.T @ D @ B
    # print(BD)
    # print(type(BD[5,5]))
    # print(BD[5,5])
    BDI=np.zeros(BD.shape)
    # BDI=area(BD)
    for i in range(BD.shape[0]):
        for j in range(BD.shape[1]):
            BDI[i,j]=integrate(BD[i,j],(s,-1,1),(t,-1,1))
    # print(BD[5,5])
    # print(BDI)
    return h*BDI
i


    



1

In [357]:
km=np.zeros((nEle,8,8))
for i in range(nEle):
    km[i]=eleStiffness(elecords,E,mu,h,i,ps=1)



[0.   0.   0.25 0.   0.25 0.25 0.   0.25]
[0.25 0.   0.5  0.   0.5  0.25 0.25 0.25]


In [358]:
np.set_printoptions(suppress=True,precision=4)
print(km/1000000)

[[[ 2.5962  0.9375 -1.5865 -0.0721 -1.2981 -0.9375  0.2885  0.0721]
  [ 0.9375  2.5962  0.0721  0.2885 -0.9375 -1.2981 -0.0721 -1.5865]
  [-1.5865  0.0721  2.5962 -0.9375  0.2885 -0.0721 -1.2981  0.9375]
  [-0.0721  0.2885 -0.9375  2.5962  0.0721 -1.5865  0.9375 -1.2981]
  [-1.2981 -0.9375  0.2885  0.0721  2.5962  0.9375 -1.5865 -0.0721]
  [-0.9375 -1.2981 -0.0721 -1.5865  0.9375  2.5962  0.0721  0.2885]
  [ 0.2885 -0.0721 -1.2981  0.9375 -1.5865  0.0721  2.5962 -0.9375]
  [ 0.0721 -1.5865  0.9375 -1.2981 -0.0721  0.2885 -0.9375  2.5962]]

 [[ 2.5962  0.9375 -1.5865 -0.0721 -1.2981 -0.9375  0.2885  0.0721]
  [ 0.9375  2.5962  0.0721  0.2885 -0.9375 -1.2981 -0.0721 -1.5865]
  [-1.5865  0.0721  2.5962 -0.9375  0.2885 -0.0721 -1.2981  0.9375]
  [-0.0721  0.2885 -0.9375  2.5962  0.0721 -1.5865  0.9375 -1.2981]
  [-1.2981 -0.9375  0.2885  0.0721  2.5962  0.9375 -1.5865 -0.0721]
  [-0.9375 -1.2981 -0.0721 -1.5865  0.9375  2.5962  0.0721  0.2885]
  [ 0.2885 -0.0721 -1.2981  0.9375 -1.5865  0.

In [359]:
K=np.zeros((2*nodes,2*nodes))
for i in range(nEle):
    for j in range(km.shape[1]):
        for k in range(km.shape[-1]):
            x=dof_label[i,k]
            y=dof_label[i,j]
            K[x,y]+=km[i][j][k]

# for i in range(nEle):
#     for j in range(km.shape[1]):
#         for k in range(km.shape[2]):
#              x=dof_label[i,k]
#              y=dof_label[i,j]
#              K[x,y]+=km[i][j][k]

In [360]:
np.set_printoptions(suppress=True,precision=2)
print(K/1000000)

[[ 2.6   0.94 -1.59 -0.07  0.    0.    0.29  0.07 -1.3  -0.94  0.    0.  ]
 [ 0.94  2.6   0.07  0.29  0.    0.   -0.07 -1.59 -0.94 -1.3   0.    0.  ]
 [-1.59  0.07  5.19  0.   -1.59 -0.07 -1.3   0.94  0.58  0.   -1.3  -0.94]
 [-0.07  0.29  0.    5.19  0.07  0.29  0.94 -1.3   0.   -3.17 -0.94 -1.3 ]
 [ 0.    0.   -1.59  0.07  2.6  -0.94  0.    0.   -1.3   0.94  0.29 -0.07]
 [ 0.    0.   -0.07  0.29 -0.94  2.6   0.    0.    0.94 -1.3   0.07 -1.59]
 [ 0.29 -0.07 -1.3   0.94  0.    0.    2.6  -0.94 -1.59  0.07  0.    0.  ]
 [ 0.07 -1.59  0.94 -1.3   0.    0.   -0.94  2.6  -0.07  0.29  0.    0.  ]
 [-1.3  -0.94  0.58  0.   -1.3   0.94 -1.59 -0.07  5.19  0.   -1.59  0.07]
 [-0.94 -1.3   0.   -3.17  0.94 -1.3   0.07  0.29  0.    5.19 -0.07  0.29]
 [ 0.    0.   -1.3  -0.94  0.29  0.07  0.    0.   -1.59 -0.07  2.6   0.94]
 [ 0.    0.   -0.94 -1.3  -0.07 -1.59  0.    0.    0.07  0.29  0.94  2.6 ]]


In [361]:
#post processing
np.set_printoptions(suppress=True,precision=5)
F=np.zeros((2*nodes,1))
U=np.zeros(np.shape(F))
GlobalF=np.zeros(np.shape(F))
GlobalU=np.zeros(np.shape(U))

F[5-1]=9.375
F[11-1]=9.375

xx=free_dof

uu=dof_seq[:xx]
F=F[dof_seq]

fp=F[:xx]

K11=K[np.ix_(uu,uu)]

u_disp=np.linalg.inv(K11) @ fp

U[:xx]=u_disp

GlobalU[dof_seq]=U
GlobalF=K@GlobalU

print(GlobalF)
print('\n')

print(GlobalU*1e5)

[[-9.375  ]
 [-1.97414]
 [-0.     ]
 [-0.     ]
 [ 9.375  ]
 [-0.     ]
 [-9.375  ]
 [ 1.97414]
 [-0.     ]
 [ 0.     ]
 [ 9.375  ]
 [ 0.     ]]


[[ 0.     ]
 [ 0.     ]
 [ 0.34395]
 [ 0.06318]
 [ 0.703  ]
 [ 0.05032]
 [ 0.     ]
 [ 0.     ]
 [ 0.34395]
 [-0.06318]
 [ 0.703  ]
 [-0.05032]]


SyntaxError: invalid syntax (Temp/ipykernel_3696/2845899866.py, line 4)

In [370]:
np.set_printoptions(suppress=False,precision=5)
u_mem=np.zeros((nEle,8))

for i in range(nEle):
    for j in range(8):
        u_mem[i][j]=GlobalU[dof_label[i,j]]

In [368]:
print(u_mem.T)

[[ 0.00000e+00  3.43954e-06]
 [ 0.00000e+00  6.31810e-07]
 [ 3.43954e-06  7.03005e-06]
 [ 6.31810e-07  5.03211e-07]
 [ 3.43954e-06  7.03005e-06]
 [-6.31810e-07 -5.03211e-07]
 [ 0.00000e+00  3.43954e-06]
 [ 0.00000e+00 -6.31810e-07]]


In [ ]:
# for i in range(nEle):
#     for j in range(8):
#         u_mem[i][j]=GlobalU[dof_label[i,j]]
  
        # zz.append(GlobalU[dof_label[i,j]])
    # u_mem[i]=np.array(zz)